### Run Multiple Scenario on Pool

In [1]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV, lang='english',check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.26.4, server:0.26.5

#### Pool

In `Pool` , it will have a field named `assets` ,which contains a list of `assets`

In [2]:
mortgage1 = ["Mortgage"
            ,{"originBalance": 12000.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 40
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 10000.0
            ,"currentRate": 0.075
            ,"remainTerm": 30
            ,"status": "current"}]

mortgage2 = ["Mortgage"
            ,{"originBalance": 800.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 25
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 600.0
            ,"currentRate": 0.075
            ,"remainTerm": 15
            ,"status": "current"}]

myPool = {'assets':[ mortgage1, mortgage2],
         'cutoffDate':"2022-03-01"}

#### Set Multi Assumption

It is just a map!

In [3]:
multiAssump= \
    {"cdr001":("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                    ,None
                                    ,None)
    ,"non":("Pool",("Mortgage",None,None,None,None)
                    ,None
                    ,None
                    )}

Make sure call `runPoolByScenarios`

In [4]:
r = localAPI.runPoolByScenarios(myPool,poolAssump=multiAssump,read=True)

Use keys from `multiAssump` to retrive results

In [10]:
r['cdr001']['PoolConsol'][0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9542.79,345.80,61.80,0,7.62,0,7.62,0.075,None,None,345.80,0,0,7.62,0,7.62
2022-04-01,9186.98,347.68,59.58,0,8.13,0,8.13,0.075,None,None,693.48,0,0,15.75,0,15.75
2022-05-01,8829.85,349.56,57.36,0,7.57,0,7.57,0.075,None,None,1043.04,0,0,23.32,0,23.32
2022-06-01,8470.88,351.45,55.13,0,7.52,0,7.52,0.075,None,None,1394.49,0,0,30.84,0,30.84
2022-07-01,8110.54,353.35,52.89,0,6.99,0,6.99,0.075,None,None,1747.84,0,0,37.83,0,37.83
2022-08-01,7748.37,355.26,50.64,0,6.91,0,6.91,0.075,None,None,2103.10,0,0,44.74,0,44.74
2022-09-01,7384.60,357.17,48.38,0,6.60,0,6.60,0.075,None,None,2460.27,0,0,51.34,0,51.34
2022-10-01,7019.39,359.12,46.10,0,6.09,0,6.09,0.075,None,None,2819.39,0,0,57.43,0,57.43
2022-11-01,6652.36,361.05,43.82,0,5.98,0,5.98,0.075,None,None,3180.44,0,0,63.41,0,63.41


#### Run Multiple Pools with Multiple Assumptions

In [11]:
receivable1 = ["Invoice"
                  ,{"start":"2024-04-01","originBalance":2000
                    ,"originAdvance":1500,"dueDate":"2024-06-01"}
                  ,{"status":"Current"}]

myMixedPool = {"invoicePool": {'assets':[receivable1],}
              ,"mortgagePool": {'assets':[mortgage1]}
              ,'cutoffDate':"2022-03-01"}

In [13]:
mPoolAssump0 = ("Pool",
               ("Mortgage",None,None,None,None)
                ,None
                ,None)

mPoolAssump = ("Pool",
               ("Mortgage",{"CDR":0.01},None,None,None)
                ,None
                ,None)

iPoolAssump0 = ("Pool"
                ,("Receivable", None, None, None)
                ,None
                ,None)

iPoolAssump = ("Pool"
                ,("Receivable", {"CDR":0.01}, None, None)
                ,None
                ,None)


r = localAPI.runPoolByScenarios(myMixedPool
                  ,poolAssump = {"NonStress":("ByPoolId",
                                    {"invoicePool":iPoolAssump0
                                    ,"mortgagePool":mPoolAssump0})
                                 ,"Stress":("ByPoolId",
                                    {"invoicePool":iPoolAssump
                                    ,"mortgagePool":mPoolAssump})}
                  ,read=True)

In [16]:
r['NonStress']['invoicePool'][0]

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-04-01,2000,0,0,0,0,0,0,0,0,0,0,0,0
2024-06-01,0,0,2000,0,0,0,0,0,0,0,0,0,0


In [17]:
r['Stress']['mortgagePool'][0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9059.53,307.14,58.54,0,7.22,0,7.22,0.075,None,None,307.14,0,0,7.22,0,7.22
2022-04-01,8743.01,308.80,56.57,0,7.72,0,7.72,0.075,None,None,615.94,0,0,14.94,0,14.94
2022-05-01,8425.32,310.48,54.59,0,7.21,0,7.21,0.075,None,None,926.42,0,0,22.15,0,22.15
2022-06-01,8105.99,312.15,52.61,0,7.18,0,7.18,0.075,None,None,1238.57,0,0,29.33,0,29.33
2022-07-01,7785.46,313.84,50.62,0,6.69,0,6.69,0.075,None,None,1552.41,0,0,36.02,0,36.02
2022-08-01,7463.28,315.54,48.61,0,6.64,0,6.64,0.075,None,None,1867.95,0,0,42.66,0,42.66
2022-09-01,7139.68,317.24,46.60,0,6.36,0,6.36,0.075,None,None,2185.19,0,0,49.02,0,49.02
2022-10-01,6814.83,318.96,44.58,0,5.89,0,5.89,0.075,None,None,2504.15,0,0,54.91,0,54.91
2022-11-01,6488.34,320.68,42.55,0,5.81,0,5.81,0.075,None,None,2824.83,0,0,60.72,0,60.72
